## Set Up Dependencies


In [ ]:
import datetime
import os

from cliffs_delta import cliffs_delta
import pandas as pd
import pecking
import matplotlib as mpl
import numpy as np
import pecking
from slugify import slugify
from teeplot import teeplot as tp

from pylib._describe_effect import describe_effect
from pylib._styler_to_pdf import styler_to_pdf


## Data Retrieval and Preprocessing


In [ ]:
df = pd.read_parquet("https://osf.io/ah7v5/download")


In [ ]:
df["Num Reconstructed Inner Nodes"] = df["num_reconstructed_inner_nodes"]
df["Num True Inner Nodes"] = df["num_true_inner_nodes"]
df["Raw Triplet Distance"] = df["triplet_distance_raw"]
df["Sampled Triplet Distance Strict Reconst"] = df[
    "sampled_triplet_distance_strict_reconst"
]
df["Strict Triplet Distance"] = df["sampled_triplet_distance_strict"]
df["Lax Triplet Distance"] = df["sampled_triplet_distance_lax"]
df["Differentia Width (bits)"] = df["differentia_width_bits"]
df["Annotation Size (bits)"] = df["annotation_size_bits"]
df["Algorithm"] = df["algorithm"]
df["Population Size"] = df["population_size"]
df["Num Generations"] = df["num_generations"]
df["Num Islands"] = df["num_islands"]
df["Num Niches"] = df["num_niches"]
df["Tournament Size"] = df["tournament_size"]
df["Replicate"] = df["replicate"]
df["Downsample"] = df["downsample"]


In [ ]:
df["Inner Node Resolution"] = (
    df["Num Reconstructed Inner Nodes"] / df["Num True Inner Nodes"]
)
df["Inner Node Loss"] = 1 - df["Inner Node Resolution"]


In [ ]:
df["Dynamics"] = (
    "islands"
    + df["Num Islands"].astype(str)
    + "-niches"
    + df["Num Niches"].astype(str)
    + "-tsize"
    + df["Tournament Size"].astype(str)
)


In [ ]:
df["Scale"] = (
    "npop"
    + df["Population Size"].astype(str)
    + "-ngen"
    + df["Num Generations"].astype(str)
)


In [ ]:
df["Scenario"] = df["Dynamics"].map(
    {
        "islands1-niches1-tsize2": "plain",
        "islands1-niches1-tsize1": "drift",
        "islands4-niches2-tsize2": "mild\nstructure",
        "islands64-niches8-tsize2": "rich\nstructure",
    }
)


In [ ]:
df["Policy"] = df["Algorithm"].map(
    {
        "surf-tilted": "Tilted",
        "col-tilted": "Tilted",
        "surf-hybrid": "Hybrid",
        "surf-steady": "Steady",
        "col-steady": "Steady",
    }
)


In [ ]:
df["Implementation"] = df["Algorithm"].map(
    {
        "surf-tilted": "Surface",
        "col-tilted": "Column",
        "surf-hybrid": "Surface",
        "surf-steady": "Surface",
        "col-steady": "Column",
    }
)


In [ ]:
for col in df.columns:
    df[col.replace(" ", "\n")] = df[col]


In [ ]:
sensitivity_analysis_variables = [
    "Differentia\nWidth\n(bits)",
    # "Algorithm",
    "Num\nGenerations",
    "Annotation\nSize\n(bits)",
    "Population\nSize",
    "Downsample",
]


In [ ]:
dfx = df.melt(
    id_vars=[
        "Algorithm",
        "Annotation Size (bits)",
        "Annotation\nSize\n(bits)",
        "annotation_size_bits",
        "Differentia Width (bits)",
        "Differentia\nWidth\n(bits)",
        "differentia_width_bits",
        "Downsample",
        "downsample",
        "Dynamics",
        "Implementation",
        "Policy",
        "Scenario",
        "Scale",
        "Population Size",
        "Population\nSize",
        "population_size",
        "Num Generations",
        "Num\nGenerations",
        "num_generations",
        "Num Islands",
        "Num Niches",
        "Tournament Size",
        "Replicate",
    ],
    value_vars=[
        "Strict\nTriplet\nDistance",
        "Lax\nTriplet\nDistance",
        "Inner\nNode\nLoss",
    ],
)
dfx.head()


## Visualize Main Metrics -- Population Size


In [ ]:
for name, group in dfx[
    (dfx["Algorithm"].isin(["surf-tilted", "surf-hybrid", "col-steady"]))
].groupby(sensitivity_analysis_variables):
    group_variables = dict(zip(sensitivity_analysis_variables, name))

    row_order = [
        "Strict\nTriplet\nDistance",
        "Lax\nTriplet\nDistance",
    ]
    if group_variables["Differentia\nWidth\n(bits)"] > 1:
        row_order.append(
            "Inner\nNode\nLoss",
        )

    tp.tee(
        pecking.peckplot,
        data=group.reset_index(drop=True),
        score="value",
        x="Scenario",
        y="value",
        # col="Num Generations",
        row="variable",
        row_order=row_order,
        hue="Policy",
        x_group="outer",
        order=[
            "plain",
            "mild\nstructure",
            "rich\nstructure",
            "drift",
        ],
        skim_hatches=("*",),
        skim_labels=("Best",),
        skimmers=[pecking.skim_lowest],
        margin_titles=True,
        height=1.7,
        aspect=2.5,
        facet_kws={
            "ylim": (0, 1),
        },
        teeplot_outattrs={
            slugify(k): slugify(v) for k, v in group_variables.items()
        },
        teeplot_outexclude="teeplot_postprocess",
        teeplot_postprocess="teed.figure.subplots_adjust(right=0.6); teed.set_titles(row_template='{row_name}')",
    )
    display(group_variables)


## Make Stat Table --- Pairwise


In [ ]:
lookup_effect = {}


def cached_delta(x, y):
    d = cliffs_delta(x, y)[0]
    lookup_effect[d] = describe_effect(x, y)
    return d


vars = [
    "Scenario",
    *sensitivity_analysis_variables,
    "Num Generations",
    "Downsample",
    "Population Size",
]

records = []
for name, group in df.groupby(vars):
    group_variables = dict(zip(vars, name))
    records.append(
        {
            **group_variables,
            **{
                "Strict<br/>Triplet<br/>Distance": cached_delta(
                    group[group["Algorithm"] == "surf-tilted"][
                        "Strict Triplet Distance"
                    ],
                    group[group["Algorithm"] == "surf-hybrid"][
                        "Strict Triplet Distance"
                    ],
                ),
                "Lax<br/>Triplet<br/>Distance": cached_delta(
                    group[group["Algorithm"] == "surf-tilted"][
                        "Lax Triplet Distance"
                    ],
                    group[group["Algorithm"] == "surf-hybrid"][
                        "Lax Triplet Distance"
                    ],
                ),
                "Inner<br/>Node<br/>Loss": (
                    cached_delta(
                        group[group["Algorithm"] == "surf-tilted"][
                            "Inner Node Loss"
                        ],
                        group[group["Algorithm"] == "surf-hybrid"][
                            "Inner Node Loss"
                        ],
                    )
                    # if group_variables["Differentia\nWidth\n(bits)"] > 1
                    # else float("nan")
                ),
            },
            "versus": "Tilted vs. Hybrid",
        }
    )

for name, group in df.groupby(vars):
    group_variables = dict(zip(vars, name))
    records.append(
        {
            **group_variables,
            **{
                "Strict<br/>Triplet<br/>Distance": cached_delta(
                    group[group["Algorithm"] == "surf-tilted"][
                        "Strict Triplet Distance"
                    ],
                    group[group["Algorithm"] == "col-steady"][
                        "Strict Triplet Distance"
                    ],
                ),
                "Lax<br/>Triplet<br/>Distance": cached_delta(
                    group[group["Algorithm"] == "surf-tilted"][
                        "Lax Triplet Distance"
                    ],
                    group[group["Algorithm"] == "col-steady"][
                        "Lax Triplet Distance"
                    ],
                ),
                "Inner<br/>Node<br/>Loss": (
                    cached_delta(
                        group[group["Algorithm"] == "surf-tilted"][
                            "Inner Node Loss"
                        ],
                        group[group["Algorithm"] == "col-steady"][
                            "Inner Node Loss"
                        ],
                    )
                    # if group_variables["Differentia\nWidth\n(bits)"] > 1
                    # else float("nan")
                ),
            },
            "versus": "Tilted vs. Steady",
        }
    )

for name, group in df.groupby(vars):
    group_variables = dict(zip(vars, name))
    records.append(
        {
            **group_variables,
            **{
                "Strict<br/>Triplet<br/>Distance": cached_delta(
                    group[group["Algorithm"] == "surf-hybrid"][
                        "Strict Triplet Distance"
                    ],
                    group[group["Algorithm"] == "col-steady"][
                        "Strict Triplet Distance"
                    ],
                ),
                "Lax<br/>Triplet<br/>Distance": cached_delta(
                    group[group["Algorithm"] == "surf-hybrid"][
                        "Lax Triplet Distance"
                    ],
                    group[group["Algorithm"] == "col-steady"][
                        "Lax Triplet Distance"
                    ],
                ),
                "Inner<br/>Node<br/>Loss": (
                    cached_delta(
                        group[group["Algorithm"] == "surf-hybrid"][
                            "Inner Node Loss"
                        ],
                        group[group["Algorithm"] == "col-steady"][
                            "Inner Node Loss"
                        ],
                    )
                    # if group_variables["Differentia\nWidth\n(bits)"] > 1
                    # else float("nan")
                ),
            },
            "versus": "Hybrid vs. Steady",
        }
    )


dfr = pd.DataFrame.from_records(records)
dfr


In [ ]:
cmap = mpl.colormaps.get_cmap("coolwarm")
cmap.set_bad(color="white")


In [ ]:
dfr["Unit"] = dfr["Differentia\nWidth\n(bits)"].map({1: "bit", 8: "byte"})
dfr["Size<br/>(bits)"] = dfr["Annotation\nSize\n(bits)"]


In [ ]:
vars = [
    "versus",
    "Scenario",
    "Unit",
    "Population\nSize",
    "Downsample",
    # "Num Generations",
    "Size<br/>(bits)",
]
styled_dfs = (
    dfr[(dfr["Num Generations"] == 100000)]
    .set_index(vars)
    .unstack(level=0)[
        [
            "Strict<br/>Triplet<br/>Distance",
            # "Lax<br/>Triplet<br/>Distance",
            "Inner<br/>Node<br/>Loss",
        ]
    ]
    .swaplevel(axis="columns")
    .sort_index(axis=1, level=0, ascending=False)
    .style.background_gradient(cmap=cmap, vmin=-1, vmax=1, axis=None)
    .format(lambda x: lookup_effect.get(x, "n/a"))
)

props = [
    ("cellpadding", "0px"),
    ("cellspacing", "0px"),
    ("border", "1px solid black"),
    ("border-collapse", "collapse"),
    ("max-width", "50px !important"),
    ("word-wrap", "break-word"),
]
moreprops = [
    *props,
    ("padding-top", "0px"),
    ("padding-bottom", "0px"),
    ("margin", "0px"),
    ("height", "0px"),
]
smallprops = [
    *moreprops,
    ("font-size", "9px"),
    ("text-align", "center"),
    ("width", "65px"),
]
bigprops = [
    *moreprops,
    ("font-size", "11px"),
    ("color", "white"),
    ("background-color", "DimGray"),
]

divider_style = [
    # Header cell borders
    {"selector": "th", "props": bigprops},
    # Data cell borders
    {"selector": "td", "props": smallprops},
    # Row borders
    {"selector": "tr", "props": smallprops},
]
styled_dfs = styled_dfs.set_table_styles(divider_style).set_table_attributes(
    'style="cellspacing:0;border-collapse:collapse;font-family:sans-serif;"'
)

styled_dfs


In [ ]:
os.makedirs("outplots", exist_ok=True)
styler_to_pdf(styled_dfs, "outplots/steady-vs-tilted-pairwise.pdf")


## Make Stat Table --- Values


In [ ]:
lookup_effect = {}


def cached_delta(x, y):
    d = cliffs_delta(x, y)[0]
    lookup_effect[d] = describe_effect(x, y)
    return d


vars = [
    "Scenario",
    *sensitivity_analysis_variables,
    "Num Generations",
    "Downsample",
    "Population Size",
]

records = []
for name, group in df[
    df["Algorithm"].isin(
        ["surf-tilted", "surf-hybrid", "col-steady"],
    )
].groupby(vars):
    group_variables = dict(zip(vars, name))
    group["Strict Triplet Distance"] = group["Strict Triplet Distance"].rank()
    group["Lax Triplet Distance"] = group["Lax Triplet Distance"].rank()
    group["Inner Node Loss"] = group["Inner Node Loss"].rank()

    std_highest = pecking.skim_highest(
        *zip(*map(reversed, group.groupby("Policy")["Strict Triplet Distance"]))
    )
    std_lowest = pecking.skim_lowest(
        *zip(*map(reversed, group.groupby("Policy")["Strict Triplet Distance"]))
    )

    ltd_highest = pecking.skim_highest(
        *zip(*map(reversed, group.groupby("Policy")["Lax Triplet Distance"]))
    )
    ltd_lowest = pecking.skim_lowest(
        *zip(*map(reversed, group.groupby("Policy")["Lax Triplet Distance"]))
    )

    inl_highest = pecking.skim_highest(
        *zip(*map(reversed, group.groupby("Policy")["Inner Node Loss"]))
    )
    inl_lowest = pecking.skim_lowest(
        *zip(*map(reversed, group.groupby("Policy")["Inner Node Loss"]))
    )

    # <span style="color: limegreen; background-color: white; font-size: 15px;"> ★ </span>

    for policy, policy_group in group.groupby("Policy", sort=False):
        records.append(
            {
                **group_variables,
                **{
                    "Strict Triplet Distance Numerical": policy_group[
                        "Strict Triplet Distance"
                    ].mean(),
                    "Lax Triplet Distance Numerical": policy_group[
                        "Lax Triplet Distance"
                    ].mean(),
                    "Inner Node Loss Numerical": (
                        policy_group["Inner Node Loss"].mean()
                        # if group_variables["Differentia\nWidth\n(bits)"] > 1
                        # else float("nan")
                    ),
                    "Strict Triplet Distance": (
                        ("<b>B</b>" * (policy in std_lowest) or " &nbsp;")
                        + "&nbsp;" * 8
                        + ("<b>w</b>" * (policy in std_highest) or " &nbsp;")
                    ),
                    # "Lax Triplet Distance": (
                    #     ("<b>B</b>" * (policy in ltd_lowest) or " &nbsp;")
                    #     + "&nbsp;" * 8
                    #     + ("<b>w</b>" * (policy in ltd_highest) or " &nbsp;")
                    # ),
                    "Inner Node Loss": (
                        (
                            ("<b>B</b>" * (policy in inl_lowest) or " &nbsp;")
                            + "&nbsp;" * 8
                            + (
                                "<b>w</b>" * (policy in inl_highest)
                                or " &nbsp;"
                            )
                        )
                        if group_variables["Differentia\nWidth\n(bits)"] > 1
                        else ""
                    ),
                    "Policy": policy,
                },
            }
        )

dfr = pd.DataFrame.from_records(records)
dfr


In [ ]:
cmap = mpl.colormaps.get_cmap("coolwarm")
cmap.set_bad(color="white")


In [ ]:
dfr["Unit"] = dfr["Differentia\nWidth\n(bits)"].map({1: "bit", 8: "byte"})
dfr["Size<br/>(bits)"] = dfr["Annotation\nSize\n(bits)"]


In [ ]:
import numpy as np

vars = [
    "Policy",
    "Scenario",
    "Unit",
    "Population\nSize",
    "Downsample",
    "Num Generations",
    "Size<br/>(bits)",
]
vals = [
    "Strict Triplet Distance",
    # "Lax Triplet Distance",
    "Inner Node Loss",
]
dfs = (
    dfr[(dfr["Num Generations"] == 100000)]
    .sort_values(
        by="Policy",
        key=np.vectorize(["Tilted", "Hybrid", "Steady"].index),
        axis=0,
    )
    .set_index(vars)
    .unstack(level=0)
    # .swaplevel(axis="columns")
    .sort_index(
        axis=1,
        level=[0, 1],
        ascending=True,
        key=np.vectorize(["Tilted", "Hybrid", "Steady", *dfr.columns].index),
    )
)

select1 = dfs.columns.get_level_values(0).isin(vals)
select2 = dfs.columns.get_level_values(0).str.contains("Numerical")
styled_dfs = dfs.loc[:, select1].style.background_gradient(
    cmap=cmap, vmin=10, vmax=51, axis=None, gmap=dfs.loc[:, select2].to_numpy()
)

props = [
    ("cellpadding", "0px"),
    ("cellspacing", "0px"),
    ("border", "1px solid black"),
    ("border-collapse", "collapse"),
    ("max-width", "50px !important"),
    ("word-wrap", "break-word"),
]
moreprops = [
    *props,
    ("padding-top", "0px"),
    ("padding-bottom", "0px"),
    ("margin", "0px"),
    ("height", "0px"),
]
smallprops = [
    *moreprops,
    ("font-size", "9px"),
    ("text-align", "center"),
    ("width", "65px"),
]
bigprops = [
    *moreprops,
    ("font-size", "11px"),
    ("color", "white"),
    ("background-color", "DimGray"),
]

divider_style = [
    # Header cell borders
    {"selector": "th", "props": bigprops},
    # Data cell borders
    {"selector": "td", "props": smallprops},
    # Row borders
    {"selector": "tr", "props": smallprops},
]
styled_dfs = styled_dfs.set_table_styles(divider_style).set_table_attributes(
    'style="cellspacing:0;border-collapse:collapse;font-family:sans-serif;"'
)

styled_dfs


In [ ]:
os.makedirs("outplots", exist_ok=True)
styler_to_pdf(styled_dfs, "outplots/steady-vs-tilted-value.pdf")


## Make Stat Table --- Surf


In [ ]:
lookup_effect = {}


def cached_delta(x, y):
    d = cliffs_delta(x, y)[0]
    lookup_effect[d] = describe_effect(x, y)
    return d


vars = [
    "Scenario",
    *sensitivity_analysis_variables,
    "Num Generations",
    "Downsample",
    "Population Size",
]

records = []
for name, group in df[
    df["Algorithm"].isin(
        ["surf-tilted", "surf-hybrid", "surf-steady"],
    )
].groupby(vars):
    group_variables = dict(zip(vars, name))
    group["Strict Triplet Distance"] = group["Strict Triplet Distance"].rank()
    group["Lax Triplet Distance"] = group["Lax Triplet Distance"].rank()
    group["Inner Node Loss"] = group["Inner Node Loss"].rank()

    std_highest = pecking.skim_highest(
        *zip(*map(reversed, group.groupby("Policy")["Strict Triplet Distance"]))
    )
    std_lowest = pecking.skim_lowest(
        *zip(*map(reversed, group.groupby("Policy")["Strict Triplet Distance"]))
    )

    ltd_highest = pecking.skim_highest(
        *zip(*map(reversed, group.groupby("Policy")["Lax Triplet Distance"]))
    )
    ltd_lowest = pecking.skim_lowest(
        *zip(*map(reversed, group.groupby("Policy")["Lax Triplet Distance"]))
    )

    inl_highest = pecking.skim_highest(
        *zip(*map(reversed, group.groupby("Policy")["Inner Node Loss"]))
    )
    inl_lowest = pecking.skim_lowest(
        *zip(*map(reversed, group.groupby("Policy")["Inner Node Loss"]))
    )

    # <span style="color: limegreen; background-color: white; font-size: 15px;"> ★ </span>

    for policy, policy_group in group.groupby("Policy", sort=False):
        records.append(
            {
                **group_variables,
                **{
                    "Strict Triplet Distance Numerical": policy_group[
                        "Strict Triplet Distance"
                    ].mean(),
                    # "Lax Triplet Distance Numerical": policy_group[
                    #     "Lax Triplet Distance"
                    # ].mean(),
                    "Inner Node Loss Numerical": (
                        policy_group["Inner Node Loss"].mean()
                        # if group_variables["Differentia\nWidth\n(bits)"] > 1
                        # else float("nan")
                    ),
                    "Strict Triplet Distance": (
                        ("<b>B</b>" * (policy in std_lowest) or " &nbsp;")
                        + "&nbsp;" * 8
                        + ("<b>w</b>" * (policy in std_highest) or " &nbsp;")
                    ),
                    # "Lax Triplet Distance": (
                    #     ("<b>B</b>" * (policy in ltd_lowest) or " &nbsp;")
                    #     + "&nbsp;" * 8
                    #     + ("<b>w</b>" * (policy in ltd_highest) or " &nbsp;")
                    # ),
                    "Inner Node Loss": (
                        (
                            ("<b>B</b>" * (policy in inl_lowest) or " &nbsp;")
                            + "&nbsp;" * 8
                            + (
                                "<b>w</b>" * (policy in inl_highest)
                                or " &nbsp;"
                            )
                        )
                        if group_variables["Differentia\nWidth\n(bits)"] > 1
                        else ""
                    ),
                    "Policy": policy,
                },
            }
        )

dfr = pd.DataFrame.from_records(records)
dfr


In [ ]:
cmap = mpl.colormaps.get_cmap("coolwarm")
cmap.set_bad(color="white")


In [ ]:
dfr["Unit"] = dfr["Differentia\nWidth\n(bits)"].map({1: "bit", 8: "byte"})
dfr["Size<br/>(bits)"] = dfr["Annotation\nSize\n(bits)"]


In [ ]:
import numpy as np

vars = [
    "Policy",
    "Scenario",
    "Unit",
    "Population\nSize",
    "Downsample",
    "Num Generations",
    "Size<br/>(bits)",
]
vals = [
    "Strict Triplet Distance",
    # "Lax Triplet Distance",
    "Inner Node Loss",
]
dfs = (
    dfr[(dfr["Num Generations"] == 100000)]
    .sort_values(
        by="Policy",
        key=np.vectorize(["Tilted", "Hybrid", "Steady"].index),
        axis=0,
    )
    .set_index(vars)
    .unstack(level=0)
    # .swaplevel(axis="columns")
    .sort_index(
        axis=1,
        level=[0, 1],
        ascending=True,
        key=np.vectorize(["Tilted", "Hybrid", "Steady", *dfr.columns].index),
    )
)

select1 = dfs.columns.get_level_values(0).isin(vals)
select2 = dfs.columns.get_level_values(0).str.contains("Numerical")
styled_dfs = dfs.loc[:, select1].style.background_gradient(
    cmap=cmap, vmin=10, vmax=51, axis=None, gmap=dfs.loc[:, select2].to_numpy()
)

props = [
    ("cellpadding", "0px"),
    ("cellspacing", "0px"),
    ("border", "1px solid black"),
    ("border-collapse", "collapse"),
    ("max-width", "50px !important"),
    ("word-wrap", "break-word"),
]
moreprops = [
    *props,
    ("padding-top", "0px"),
    ("padding-bottom", "0px"),
    ("margin", "0px"),
    ("height", "0px"),
]
smallprops = [
    *moreprops,
    ("font-size", "9px"),
    ("text-align", "center"),
    ("width", "65px"),
]
bigprops = [
    *moreprops,
    ("font-size", "11px"),
    ("color", "white"),
    ("background-color", "DimGray"),
]

divider_style = [
    # Header cell borders
    {"selector": "th", "props": bigprops},
    # Data cell borders
    {"selector": "td", "props": smallprops},
    # Row borders
    {"selector": "tr", "props": smallprops},
]
styled_dfs = styled_dfs.set_table_styles(divider_style).set_table_attributes(
    'style="cellspacing:0;border-collapse:collapse;font-family:sans-serif;"'
)

styled_dfs


In [ ]:
os.makedirs("outplots", exist_ok=True)
styler_to_pdf(styled_dfs, "outplots/steady-vs-tilted-surf.pdf")


## Reproducibility


In [ ]:
datetime.datetime.now().isoformat()


In [ ]:
%load_ext watermark
%watermark


In [ ]:
!pip freeze


In [ ]:
!inkscape --version
